In [49]:
import numpy as np
import cv2
from tkinter import Tk, filedialog
from pathlib import Path
import os
from PIL import Image
import PIL.Image
import pytesseract

import matplotlib.pyplot as plt

In [2]:
def initiate_tk_dialog():
    root = Tk()  # pointing root to Tk() to use it as Tk() in program.
    root.withdraw()  # Hides small tkinter window.
    root.attributes('-topmost', True)  # Opened windows will be active. above all windows despite of selection.
    return root

def get_data_file(title: str, as_path = False):
    root = initiate_tk_dialog()
    if as_path == True:
        data_file: Path = Path(filedialog.askopenfilename(title= title))  # Returns opened path as str
    else:
        data_file: str = str(filedialog.askopenfilename(title= title)) 
    root.destroy()
    return data_file

def get_directory(title: str, as_path = False):
    root = initiate_tk_dialog()
    if as_path == True:
        directory: Path = Path(filedialog.askdirectory(title= title))
    else:
        directory: str = str(filedialog.askdirectory(title= title))
    root.destroy()
    return directory

In [32]:
# import cv2
# from pytesseract import image_to_string
# import numpy as np

def getText(filename):
    img = cv2.imread(filename)
    HSV_img = cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
    h,s,v = cv2.split(HSV_img)
    v = cv2.GaussianBlur(v, (1,1), 0)
    thresh = cv2.threshold(v, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
    cv2.imwrite('{}.png'.format(filename),thresh)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, ksize=(1, 2))
    thresh = cv2.dilate(thresh, kernel)
    txt = pytesseract.image_to_string(thresh, config="--psm 6 digits")
    return txt
    

# text = getText('WYOtF.png')
# print(text)
# text = getText('0Oqfr.png')
# print(text)



In [77]:
img = cv2.imread(get_data_file('Get Image', as_path=False))
HSV_img = cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
h,s,v = cv2.split(HSV_img)
v = cv2.GaussianBlur(v, (1,1), 0)
thresh = cv2.threshold(v, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
cv2.imwrite('{}.png'.format(filename),thresh)
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, ksize=(1, 2))
thresh = cv2.dilate(thresh, kernel)
txt = pytesseract.image_to_string(thresh)#, config="--psm 6 digits")

In [78]:
txt

'2022-\noak\n2002-1\n2022-1\n\n3622\n\n6-25\n5-26\n2-26\n06-25\nos 28\n\nan\nu\n11\niL\n\n02 236:\n02286\n2:53\n03:01\n03.08.\n\nat\na\n\npad\n423\nS13\n\n0.988127\n@.aa1nad\n9.aa1977\no.aazvis\n9 2a289q\n'

In [76]:
img = get_data_file('Get Image', as_path=False)
img = PIL.Image.open(img).convert("RGB")
pytesseract.pytesseract.tesseract_cmd = Path(r'C:\Program Files\Tesseract-OCR\tesseract.exe')
txt = pytesseract.image_to_string(img,
                           lang='eng',
                           timeout=5)
print(txt)

@.440127
e.ea10a3
0.401977

Npeeitort



In [70]:
txt.encode(encoding = 'UTF-8', errors = 'strict')

b'.988327\n@.\xc2\xa2a1043\n0.441977\n\n9 443896\n'

In [79]:
txt = txt.split(sep='\n\n')

In [80]:
empty_list = []
for n in txt:
    empty_list.append(n.split('\n'))

In [81]:
empty_list

[['2022-', 'oak', '2002-1', '2022-1'],
 ['3622'],
 ['6-25', '5-26', '2-26', '06-25', 'os 28'],
 ['an', 'u', '11', 'iL'],
 ['02 236:', '02286', '2:53', '03:01', '03.08.'],
 ['at', 'a'],
 ['pad', '423', 'S13'],
 ['0.988127', '@.aa1nad', '9.aa1977', 'o.aazvis', '9 2a289q', '']]

In [21]:
img = get_data_file('Get Image', as_path=False)
img = cv2.imread("input_image.png", 0)
ret, thresh = cv2.threshold(img, 10, 255, cv2.THRESH_OTSU)

In [28]:
directory = get_directory('Select Save Directory', as_path= False)
filename = directory + '\output.png'
cv2.imwrite(filename, img[thresh])

TypeError: 'NoneType' object is not subscriptable

In [42]:
img_path = get_data_file('Select File', as_path= False)
img = cv2.imread(img_path)
convert_bin,grey_scale = cv2.threshold(img,128,255,cv2.THRESH_BINARY | cv2.THRESH_OTSU)
grey_scale = 255-grey_scale
grey_graph = plt.imshow(grey_scale,cmap='gray')
plt.show()

error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\thresh.cpp:1555: error: (-2:Unspecified error) in function 'double __cdecl cv::threshold(const class cv::_InputArray &,const class cv::_OutputArray &,double,double,int)'
> THRESH_OTSU mode:
>     'src_type == CV_8UC1 || src_type == CV_16UC1'
> where
>     'src_type' is 16 (CV_8UC3)


In [43]:
img_path = get_data_file('Select File', as_path= False)
getText(img_path)

'182022-06-221614191.104-860\n1202022-06-22-16141206391.27278-068\n200202-228\n\n49202208-2216422-7261-286608\n60.2022-06-22-16-41-33-7751.274540-06\n'

In [46]:
#Function to Extract Text
def TxtExtract(directory):
    """
    This function will handle the core OCR processing of images.
    """
    
    for subdir, dirs, files in os.walk(directory):
        for file in files:
            filepath = subdir + os.sep + file
            text = pytesseract.image_to_string(Image.open(filepath), timeout=5)
            if not text:
                ex_txt.extend([[file, "blank"]])
            else:   
                ex_txt.extend([[file, text]])
                
    fol_nm = os.path.split(os.path.dirname(subdir))[-1]
    
    print(f"Text Extracted from the files in '{fol_nm}' folder & saved to list..")

In [51]:
ex_txt = []   #list to store the extracted text
TxtExtract(directory)

Text Extracted from the files in '01_Cuttings Sensor' folder & saved to list..
